In [ ]:
import json
import pandas as pd
import os
import tqdm

# 筛选课程

In [ ]:
need_course = pd.read_pickle(f'data/entities/course.pkl')
need_course['id'] = list(need_course['id'].apply(lambda x: int(x.split('_')[1])))
need_course = need_course.reset_index(drop=True)
ALL_C_id = list(need_course['id'])
print(need_course['name'].value_counts())
need_course = need_course.sort_values(by='name', ascending=False)
need_course.head()

## 展开所有的资源

In [ ]:
all_re_content = []
for in_x, one_course in need_course.iterrows():
    one_c_id = one_course['id']
    one_c_name = one_course['name']
    one_re_list = one_course['resource']
    one_c_735164 = []
    for one_re in one_re_list:
        one_tit = one_re['titles']
        one_re_id = one_re['resource_id'].split('_')
        re_type = one_re_id[0]
        kp_1 = one_tit[0]
        kp_2 = one_tit[1]
        kp_3 = one_tit[2]
        one_re_content = {'course_id':one_c_id,'re_id':int(one_re_id[1]),'re_type':re_type,
                          'title_old':one_tit,
                          'kp_1':kp_1,'kp_2':kp_2,'kp_3':kp_3,
                         'chapter':one_re['chapter'],'c_name':one_c_name}
        all_re_content.append(one_re_content)
all_re_content = pd.DataFrame(all_re_content)
ALL_EXM_id = list(all_re_content[all_re_content['re_type']=='Ex']['re_id'])
ALL_VIDEO_id = list(all_re_content[all_re_content['re_type']=='V']['re_id'])
print(len(ALL_EXM_id),'exms')
print(len(ALL_VIDEO_id),'video')
all_re_content.head()

## 拿到所有的练习id，找对应的问题id

In [ ]:
exercise_problem = pd.read_pickle(f'data/relations/exercise_problem.pkl')
exercise_problem = exercise_problem[exercise_problem['e_id'].isin(ALL_EXM_id)]
ALL_Q_id = list(exercise_problem['q_id'])
print(len(ALL_Q_id),'questions')
print(len(ALL_EXM_id),'exms')
print(np.mean(exercise_problem['e_id'].value_counts()))
exercise_problem.head()
exercise_problem.to_pickle('data/select_data/exercise_problem.pkl')

# 利用课程找学生

In [ ]:
user = pd.read_pickle(f'data/entities/user.pkl')
user['id'] = user['id'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
user = user.reset_index(drop=True)
ALL_STU_id = list(user['id'])
print(len(ALL_STU_id),'students')
user.head()

# 获取观看过视频的学生

In [ ]:
"""只运行一次"""
user_video = pd.read_pickle(f'data/relations/user-video.pkl')
user_video['user_id'] = user_video['user_id'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
all_need_user_video = []
for i_x,one_row in user_video.iterrows():
    for one_seq in one_row['seq']:
        one_data = one_seq.copy()
        for one_p in one_data['segment']:
            one_p_data = one_p.copy()
            one_p_data['user_id'] = one_row['user_id']
            one_p_data['video_id'] = one_data['video_id']
            all_need_user_video.append(one_p_data)
all_need_user_video = pd.DataFrame(all_need_user_video)
all_need_user_video['video_id'] = all_need_user_video['video_id'].apply(lambda x: int(x.split('_')[1]))
all_need_user_video

In [ ]:
ALL_Video_data = all_re_content[all_re_content['re_type']=='V']
video_2_course = dict(zip(ALL_Video_data['re_id'],ALL_Video_data['course_id']))
all_need_user_video['course_id'] = all_need_user_video['video_id'].map(video_2_course)
course_cilick = all_need_user_video['course_id'].value_counts()
good_course = list(course_cilick[course_cilick>10000].index)

# 利用学生和课程，筛选学生的作答数据

In [ ]:
all_user_problem = pd.read_pickle(f'data/relations/all_user_problem.pkl')
all_user_problem.head()

In [ ]:
ALL_Ex_data = all_re_content[all_re_content['re_type']=='Ex']
ex_2_course = dict(zip(ALL_Ex_data['re_id'],ALL_Ex_data['course_id']))
q_2_ex = dict(zip(exercise_problem['q_id'],exercise_problem['e_id']))
all_user_problem['ex_id'] = all_user_problem['problem_id'].map(q_2_ex)
all_user_problem['course_id'] = all_user_problem['ex_id'].map(ex_2_course)
good_course_q = all_user_problem['course_id'].value_counts()
good_course_id = list(good_course_q[good_course_q>100000].index)
select_course = list(set(good_course_id) & set(good_course))
select_course_q_data = all_user_problem[all_user_problem['course_id'].isin(select_course)]
select_course_video_data = all_need_user_video[all_need_user_video['course_id'].isin(select_course)]
select_course_video_data

In [ ]:
video_stu_num = select_course_video_data['user_id'].value_counts()
video_stu_num = video_stu_num[video_stu_num>10].index
q_stu_num = select_course_q_data['user_id'].value_counts()
q_stu_num = q_stu_num[q_stu_num>50].index
select_stu = list(set(video_stu_num) & set(q_stu_num))
print(len(select_stu))

In [ ]:
final_user_video_data = select_course_video_data[select_course_video_data['user_id'].isin(select_stu)]
final_user_video_data = final_user_video_data.reset_index(drop=True)
final_user_video_data['ts'] = pd.to_datetime(final_user_video_data['local_start_time'], unit='s')
final_user_video_data.head()

In [ ]:
final_user_question_data = select_course_q_data[select_course_q_data['user_id'].isin(select_stu)]
need_col = ['problem_id', 'user_id', 'is_correct', 'submit_time', 'ex_id', 'course_id']
final_user_question_data = final_user_question_data.reset_index(drop=True)[need_col]
final_user_question_data.head()

In [ ]:
final_course_data = need_course[need_course['id'].isin(select_course)]
need_col = ['id','name','about','resource']
final_user_data = user[user['id'].isin(select_stu)]
final_user_data = final_user_data.reset_index(drop=True)
final_user_data.to_pickle('data/select_data/user.pkl')
final_user_question_data.to_pickle('data/select_data/user_question.pkl')
final_user_video_data.to_pickle('data/select_data/user_video.pkl')
final_course_data.to_pickle('data/select_data/course.pkl')

# 获取学生的评论数据

In [ ]:
all_comment = pd.read_pickle('data/entities/comment.pkl')
all_comment['id']  = all_comment['id'].apply(lambda x: int(x.split('_')[1]))
all_comment = all_comment[all_comment['user_id'].isin(select_stu)]
all_comment.to_pickle('data/select_data/user_comment.pkl')
all_comment

In [ ]:
concept_video = pd.read_pickle('data/relations/concept_video.pkl')
concept_video